# Permits:
Due to the difficulty of getting the geopandas module to work, this notebook will focus on cleaning up dates out of range, non-residential properties in the permits, and calculating approval time

### would be good to turn these into functions which can be applied

In [2]:
import pandas as pd
import numpy as np
import datetime
import pandas_usaddress

pd.set_option("display.max_columns",None)

def trim_zeros(st):
    if isinstance(st,str):
        if st == '0000':
            return ''
        return st.lstrip('0')
    else:
        return None

In [16]:
df = pd.read_csv('Building_Permits.csv', low_memory=False)

In [17]:
df.columns

Index(['Permit Number', 'Permit Type', 'Permit Type Definition',
       'Permit Creation Date', 'Block', 'Lot', 'Street Number',
       'Street Number Suffix', 'Street Name', 'Street Suffix', 'Unit',
       'Unit Suffix', 'Description', 'Current Status', 'Current Status Date',
       'Filed Date', 'Issued Date', 'Completed Date',
       'First Construction Document Date', 'Structural Notification',
       'Number of Existing Stories', 'Number of Proposed Stories',
       'Voluntary Soft-Story Retrofit', 'Fire Only Permit',
       'Permit Expiration Date', 'Estimated Cost', 'Revised Cost',
       'Existing Use', 'Existing Units', 'Proposed Use', 'Proposed Units',
       'Plansets', 'TIDF Compliance', 'Existing Construction Type',
       'Existing Construction Type Description', 'Proposed Construction Type',
       'Proposed Construction Type Description', 'Site Permit',
       'Supervisor District', 'Neighborhoods - Analysis Boundaries', 'Zipcode',
       'Location', 'Record ID', 'SF Fi

In [18]:
# drop useless columns
del_cols = df.iloc[:,-5:].columns
df.drop(del_cols, axis=1, inplace=True)

### First, restrict the dataset to the last 30 years

In [5]:
df['Permit Creation Date'] = pd.to_datetime(df['Filed Date'])
df.set_index('Permit Creation Date', inplace = True)

In [6]:
# July12007 to June302018
# also need to restrict projects that aren't finished yet?
# 
df_permits = df[(df.index >= datetime.datetime(2007,7,1)) & (df.index <= datetime.datetime(2018,6,30))] 

In [7]:
df_permits['Existing Use'].value_counts().head(15)

1 family dwelling       97609
apartments              77081
office                  46791
2 family dwelling       46108
retail sales            14213
food/beverage hndlng    10275
tourist hotel/motel      3206
vacant lot               2264
warehouse,no frnitur     1612
residential hotel        1588
school                   1383
lending institution      1205
clinics-medic/dental     1066
church                   1000
artist live/work          804
Name: Existing Use, dtype: int64

### Restrict the dataset to only dwellings and apartments

In [8]:
residences = ['1 family dwelling','apartments','2 family dwelling']
df_res = pd.concat([df_permits.groupby('Existing Use').get_group(r) for r in residences])

In [9]:
df_res['Existing Use'].value_counts()

1 family dwelling    97609
apartments           77081
2 family dwelling    46108
Name: Existing Use, dtype: int64

In [10]:
df_res.shape

(220798, 46)

In [11]:
# percemtage of permits with a a valid issue date
df_res['Issued Date'].notna().sum()/df_res['Issued Date'].shape[0]


0.9575403762715242

### approval time(days)


In [12]:
df_res['Filed Date'] = df_res['Filed Date'].apply(pd.to_datetime)
df_res['Issued Date'] = df_res['Issued Date'].apply(pd.to_datetime)


In [13]:
df_res['approval_in_days'] = df_res['Issued Date'] - df_res['Filed Date']

In [14]:
df_res['approval_in_days'].dropna().describe()

count                      211423
mean      32 days 17:25:28.403248
std      126 days 01:23:19.318148
min               0 days 00:00:00
25%               0 days 00:00:00
50%               0 days 00:00:00
75%               4 days 00:00:00
max            4472 days 00:00:00
Name: approval_in_days, dtype: object

### Clean Address in Series

In [16]:
df_res['Street Number Suffix'] = df_res['Street Number Suffix'].replace(np.NaN,'')
df_res['Street Suffix'] = df_res['Street Suffix'].replace(np.NaN,'')
df_res['Unit'] = df_res['Unit'].fillna(0).apply(lambda x: str(int(x)) if (int(x)>0) else '')

In [31]:
df_res['Unit']

NameError: name 'df_res' is not defined

In [17]:

df_res['Address'] = (df_res['Street Number'].astype('str') + df_res['Street Number Suffix'].astype('str')
                    + ' ' + df_res['Street Name'].astype('str')  + ' ' + df_res['Street Suffix'].astype('str')
                    + ' ' + df_res['Unit'].astype('str') + ' ' + df_res['Unit Suffix'].astype('str')
                    )
                    
df_res['Address'] = df_res['Address'].apply(lambda x: " ".join(x.strip(' nan').strip(' nan').split()))

In [18]:
df_res['Address'][df_res['Unit Suffix'].notna()] #checking the appended number suffixes: there WILL BE SOME ISSUES ~1%

Permit Creation Date
2007-07-17             1420 La Playa A
2007-09-05            422 Vallejo St A
2007-09-17           1344 Alabama St B
2007-09-27              1575 22nd Av F
2007-10-19          292 Arlington St F
                        ...           
2018-04-11      4503 18th St RES UPPER
2018-04-13    1269 South Van Ness Av D
2018-05-16          1619 48th Av UPPER
2018-06-04           140 Arguello Bl A
2018-06-06           222 Winfield St A
Name: Address, Length: 3195, dtype: object

In [19]:
df_res.columns

Index(['Permit Number', 'Permit Type', 'Permit Type Definition', 'Block',
       'Lot', 'Street Number', 'Street Number Suffix', 'Street Name',
       'Street Suffix', 'Unit', 'Unit Suffix', 'Description', 'Current Status',
       'Current Status Date', 'Filed Date', 'Issued Date', 'Completed Date',
       'First Construction Document Date', 'Structural Notification',
       'Number of Existing Stories', 'Number of Proposed Stories',
       'Voluntary Soft-Story Retrofit', 'Fire Only Permit',
       'Permit Expiration Date', 'Estimated Cost', 'Revised Cost',
       'Existing Use', 'Existing Units', 'Proposed Use', 'Proposed Units',
       'Plansets', 'TIDF Compliance', 'Existing Construction Type',
       'Existing Construction Type Description', 'Proposed Construction Type',
       'Proposed Construction Type Description', 'Site Permit',
       'Supervisor District', 'Neighborhoods - Analysis Boundaries', 'Zipcode',
       'Location', 'Record ID', 'SF Find Neighborhoods',
       'Curr

In [20]:
#pickle the dataframe for later
# path_pickle = './Pickled/df_res_use_addr.pkl'
# df_res.to_pickle(path_pickle)

### Cleaned(ish) permits data

In [3]:
path_pickle = './Pickled/df_res_use_addr.pkl'
df_permits = pd.read_pickle(path_pickle)

In [5]:
df_permits['Permit Type Definition'].value_counts()

otc alterations permit              202368
additions alterations or repairs     17799
demolitions                            404
sign - erect                           226
new construction wood frame              1
Name: Permit Type Definition, dtype: int64

### Assessor Data
Splitting the address and reassembling them in a way that (hopefully?) makes sense

split the property location into : 
'Street Number', 'Street Number Suffix', 'Street Name',
       'Street Suffix', 'Unit', 'Unit Suffix'

In [23]:
df_assess = pd.read_csv('./Assessor Notebooks/Residence Assessor.csv')

/home/jovyan/conda-envs/capstone-dev/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (27) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [24]:
df_assess['Property Location'].head(20)

0      0000 0297 BALTIMORE           WY0000
1      0000 0666 POST                ST1503
2      0000 0650 CHESTNUT            ST0202
3      0000 0611 WASHINGTON          ST2402
4      0000 0000                       0000
5      0000 1925 LEAVENWORTH         ST0012
6      0000 0849 MASON               ST0001
7     0000 0690 CHESTNUT            ST0202C
8      0000 2001 GRANT               AV0000
9     0000 0044 MACONDRAY           LN0006E
10     0000 0006 SHARP               PL0000
11     0000 0850 POWELL              ST0301
12     0000 0240 LOMBARD             ST0289
13     0000 1520 TAYLOR              ST0205
14     0000 0850 POWELL              ST0903
15     0156 0150 LOMBARD             ST0027
16     0000 0999 GREEN               ST3101
17     0000 1001 PINE                ST1412
18    0000 0101 LOMBARD             ST0801W
19     0000 0666 FILBERT             ST0000
Name: Property Location, dtype: object

In [25]:
def parse_assess_address(series):
    series.apply(lambda x: x.split())
    print(series.apply(lambda x: x.split()))

In [26]:
parse_assess_address(df_assess['Property Location'].head())

0     [0000, 0297, BALTIMORE, WY0000]
1          [0000, 0666, POST, ST1503]
2      [0000, 0650, CHESTNUT, ST0202]
3    [0000, 0611, WASHINGTON, ST2402]
4                  [0000, 0000, 0000]
Name: Property Location, dtype: object


### Address cleaned - Permits


In [15]:
df_permits.shape

(220798, 48)

In [16]:
# altneratively, can try to standardize to s asingle address
addr_cols = ['Street Number', 'Street Name','Street Suffix', 'Unit']

df_tagged_single = pandas_usaddress.tag(df_permits,addr_cols, granularity='single', standardize=True)

In [47]:
df_tagged_single.head()

,Permit Number,Permit Type,Permit Type Definition,Block,Lot,Street Number,Street Number Suffix,Street Name,Street Suffix,Unit,...,Zipcode,Location,Record ID,SF Find Neighborhoods,Current Police Districts,Current Supervisor Districts,Analysis Neighborhoods,approval_in_days,Address,SingleLine
Permit Creation Date,,,,,,,,,,,,,,,,,,,,,
2014-02-03,201402037639,8,otc alterations permit,4219,043,1123,NaN,Carolina,St,NaN,...,94107.0,"(37.754306947688534, -122.39931683984678)",1331529172376,54.0,2.0,9.0,26.0,NaN,1123 Carolina St,1123 carolina st
2017-12-18,201712186766,3,additions alterations or repairs,6940,031,242,NaN,Ashton,Av,NaN,...,94112.0,"(37.7226331255751, -122.46195088679892)",1491022214718,64.0,10.0,8.0,24.0,NaN,242 Ashton Av,242 ashton ave
2016-08-29,201608296300,3,additions alterations or repairs,7089,040,274,NaN,Victoria,St,NaN,...,94132.0,"(37.71566908934369, -122.46503924892617)",1435480220959,65.0,10.0,1.0,24.0,NaN,274 Victoria St,274 victoria st
2015-06-02,201506027829,3,additions alterations or repairs,5841,005,423,NaN,Justin,Dr,NaN,...,94110.0,"(37.73195576909413, -122.42389453610747)",1383635189245,93.0,9.0,2.0,2.0,NaN,423 Justin Dr,423 justin dr
2016-12-21,201612215583,3,additions alterations or repairs,0155,013,1310,NaN,Pacific,Av,NaN,...,94109.0,"(37.79576706146776, -122.41710075064503)",144822060927,16.0,6.0,3.0,32.0,NaN,1310 Pacific Av,1310 pacific ave


In [48]:
df_tagged_single['SingleLine']

Permit Creation Date
2014-02-03       1123 carolina st
2017-12-18         242 ashton ave
2016-08-29        274 victoria st
2015-06-02          423 justin dr
2016-12-21       1310 pacific ave
                     ...         
2016-05-27           4971 17th st
2015-12-15    1270 fitzgerald ave
2018-02-22    1272 fitzgerald ave
2018-03-22         3533 market st
2009-02-20         684 sanchez st
Name: SingleLine, Length: 220798, dtype: object

### Load Clean Permits

In [3]:
path_pickle = './Pickled/df_addr_clean.pkl'
# df_tagged_single.to_pickle(path_pickle)
df_permits_clean = pd.read_pickle(path_pickle)

In [4]:
df_permits_clean.SingleLine.head(5)

Permit Creation Date
2014-02-03    1123 carolina st
2017-12-18      242 ashton ave
2016-08-29     274 victoria st
2015-06-02       423 justin dr
2016-12-21    1310 pacific ave
Name: SingleLine, dtype: object

### Fixing Assessors and merging with pandas usaaddress
`granularity='single', standardize=True)`

In [2]:
df_assessor = pd.read_csv('Assessor_Notebooks/Residence_Assessor.csv')

/home/jovyan/conda-envs/capstone-dev/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (27) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df_assessor.rename(columns={'Unnamed: 0':'assess_id'}, inplace=True)

In [4]:
dropcols = ['Supervisor District', 'Analysis Neighborhood',
       'the_geom', 'Row ID', 'SF Find Neighborhoods',
       'Current Police Districts', 'Current Supervisor Districts',
       'Analysis Neighborhoods',]
df_assessor.drop(dropcols, axis=1, inplace=True)
df_assessor.columns

Index(['assess_id', 'Closed Roll Year', 'Property Location', 'Parcel Number',
       'Block', 'Lot', 'Volume Number', 'Use Code', 'Use Definition',
       'Property Class Code', 'Property Class Code Definition',
       'Year Property Built', 'Number of Bathrooms', 'Number of Bedrooms',
       'Number of Rooms', 'Number of Stories', 'Number of Units',
       'Zoning Code', 'Construction Type', 'Lot Depth', 'Lot Frontage',
       'Property Area', 'Basement Area', 'Lot Area', 'Lot Code',
       'Tax Rate Area Code', 'Percent of Ownership', 'Exemption Code',
       'Exemption Code Definition', 'Status Code', 'Misc Exemption Value',
       'Homeowner Exemption Value', 'Current Sales Date',
       'Assessed Fixtures Value', 'Assessed Improvement Value',
       'Assessed Land Value', 'Assessed Personal Property Value',
       'Assessor Neighborhood District', 'Assessor Neighborhood Code',
       'Assessor Neighborhood'],
      dtype='object')

In [5]:
# 'Lot', 'Street Number', 'Street Number Suffix', 'Street Name',
#        'Street Suffix', 'Unit', 'Unit Suffix'

In [37]:
df_assessor['Lot'] = df_assessor['Property Location'].str.extract(r'(\d\d\d\d\s)', expand=True)
df_assessor['Street Number'] = df_assessor['Property Location'].str.extract(r'(?:\d\d\d\d\s)(\d\d\d\d\s)', expand=True)
df_assessor['Street Number'] = df_assessor['Street Number'].apply(trim_zeros)
df_assessor['Street Name'] = df_assessor['Property Location'].str.extract(r'(?:\d\d\d\d\s)(?:\d\d\d\d\s+)(.*)(?:\s+\w\w\d\d\d\d)', expand=True)
df_assessor['Street Suffix'] = df_assessor['Property Location'].str.extract(r'(?:\d\d\d\d\s)(?:\d\d\d\d\s+)(?:.*)(?:\s+)(\w\w)(?:\d\d\d\d)', expand=True)
df_assessor['Unit'] = df_assessor['Property Location'].str.extract(r'(?:\d\d\d\d\s)(?:\d\d\d\d\s+)(?:.*)(?:\s+)(?:\w\w)(\d\d\d\d)', expand=True)
df_assessor['Unit'] = df_assessor['Unit'].apply(trim_zeros)
df_assessor.head(3)

,assess_id,Closed Roll Year,Property Location,Parcel Number,Block,Lot,Volume Number,Use Code,Use Definition,Property Class Code,...,Assessed Improvement Value,Assessed Land Value,Assessed Personal Property Value,Assessor Neighborhood District,Assessor Neighborhood Code,Assessor Neighborhood,Street Number,Street Name,Street Suffix,Unit
0,3,2009.0,0000 0297 BALTIMORE WY0000,6499040,6499,0000,38,SRES,Single Family Residential,D,...,45734,22044,0,10.0,10B,Croker Amazon,297,BALTIMORE,WY,
1,8,2007.0,0000 0666 POST ST1503,0298C050,0298C,0000,3,SRES,Single Family Residential,COS,...,143223,35803,0,8.0,8A,Downtown,666,POST,ST,1503
2,9,2007.0,0000 0650 CHESTNUT ST0202,0051091,0051,0000,1,SRES,Single Family Residential,Z,...,284690,122009,0,8.0,8D,North Beach,650,CHESTNUT,ST,202


In [38]:
# data set is too large: separate the ID, address columns, them add the standardized address to the original
addr_cols = ['Street Number', 'Street Name','Street Suffix', 'Unit']
addr_only = df_assessor[addr_cols]
addr_only.head()

,Street Number,Street Name,Street Suffix,Unit
0,297,BALTIMORE,WY,
1,666,POST,ST,1503
2,650,CHESTNUT,ST,202
3,611,WASHINGTON,ST,2402
4,,NaN,NaN,None


In [39]:
addr_cols = ['Street Number', 'Street Name','Street Suffix', 'Unit']
std_addresses = pandas_usaddress.tag(addr_only, addr_cols, granularity='single', standardize=True)
std_addresses.head()

,Street Number,Street Name,Street Suffix,Unit,SingleLine
0,297,BALTIMORE,WY,NaN,297 baltimore way
1,666,POST,ST,1503,666 post st 1503
2,650,CHESTNUT,ST,202,650 chestnut st 202
3,611,WASHINGTON,ST,2402,611 washington st 2402
4,,NaN,NaN,NaN,NaN


In [40]:
#JOIN the addresses by the index to make df_assessor_tagged
df_assessor['SingleLine'] = std_addresses['SingleLine']
df_assessor.head(2)

,assess_id,Closed Roll Year,Property Location,Parcel Number,Block,Lot,Volume Number,Use Code,Use Definition,Property Class Code,...,Assessed Land Value,Assessed Personal Property Value,Assessor Neighborhood District,Assessor Neighborhood Code,Assessor Neighborhood,Street Number,Street Name,Street Suffix,Unit,SingleLine
0,3,2009.0,0000 0297 BALTIMORE WY0000,6499040,6499,0000,38,SRES,Single Family Residential,D,...,22044,0,10.0,10B,Croker Amazon,297,BALTIMORE,WY,,297 baltimore way
1,8,2007.0,0000 0666 POST ST1503,0298C050,0298C,0000,3,SRES,Single Family Residential,COS,...,35803,0,8.0,8A,Downtown,666,POST,ST,1503,666 post st 1503


### Load Clean Assessments

In [7]:
path_pickle = './Pickled/df_assess_clean.pkl'
# df_assessor.to_pickle(path_pickle)
df_assess_clean = pd.read_pickle(path_pickle)

In [8]:
df_assess_clean.head(1)

,assess_id,Closed Roll Year,Property Location,Parcel Number,Block,Lot,Volume Number,Use Code,Use Definition,Property Class Code,...,Assessed Land Value,Assessed Personal Property Value,Assessor Neighborhood District,Assessor Neighborhood Code,Assessor Neighborhood,Street Number,Street Name,Street Suffix,Unit,SingleLine
0,3,2009.0,0000 0297 BALTIMORE WY0000,6499040,6499,0000,38,SRES,Single Family Residential,D,...,22044,0,10.0,10B,Croker Amazon,297,BALTIMORE,WY,,297 baltimore way


### To merge, fix year to datetime (assessments)

In [9]:
df_assess_clean.rename(columns={'Closed Roll Year':'Year'},inplace=True)

In [22]:
df_assess_clean['Year'] = pd.to_datetime(df_assess_clean['Year'], format='%Y').dt.year
df_assess_clean['Year']= df_assess_clean['Year'].fillna(-1).astype(int)

In [23]:
df_assess_clean.loc[:,['Year', 'SingleLine']]

,Year,SingleLine
0,2009,297 baltimore way
1,2007,666 post st 1503
2,2007,650 chestnut st 202
3,2007,611 washington st 2402
4,2007,NaN
...,...,...
1783686,2016,588 mission bay north bl
1783687,2016,1180 4th st
1783688,2016,1180 4th st
1783689,2016,1000 3rd st


### To merge, split year off from datetime

In [24]:
df_permits_clean['Year'] = df_permits_clean.index.year

In [25]:
df_permits_clean.head()

,Permit Number,Permit Type,Permit Type Definition,Block,Lot,Street Number,Street Number Suffix,Street Name,Street Suffix,Unit,...,Location,Record ID,SF Find Neighborhoods,Current Police Districts,Current Supervisor Districts,Analysis Neighborhoods,approval_in_days,Address,SingleLine,Year
Permit Creation Date,,,,,,,,,,,,,,,,,,,,,
2014-02-03,201402037639,8,otc alterations permit,4219,043,1123,NaN,Carolina,St,NaN,...,"(37.754306947688534, -122.39931683984678)",1331529172376,54.0,2.0,9.0,26.0,NaN,1123 Carolina St,1123 carolina st,2014
2017-12-18,201712186766,3,additions alterations or repairs,6940,031,242,NaN,Ashton,Av,NaN,...,"(37.7226331255751, -122.46195088679892)",1491022214718,64.0,10.0,8.0,24.0,NaN,242 Ashton Av,242 ashton ave,2017
2016-08-29,201608296300,3,additions alterations or repairs,7089,040,274,NaN,Victoria,St,NaN,...,"(37.71566908934369, -122.46503924892617)",1435480220959,65.0,10.0,1.0,24.0,NaN,274 Victoria St,274 victoria st,2016
2015-06-02,201506027829,3,additions alterations or repairs,5841,005,423,NaN,Justin,Dr,NaN,...,"(37.73195576909413, -122.42389453610747)",1383635189245,93.0,9.0,2.0,2.0,NaN,423 Justin Dr,423 justin dr,2015
2016-12-21,201612215583,3,additions alterations or repairs,0155,013,1310,NaN,Pacific,Av,NaN,...,"(37.79576706146776, -122.41710075064503)",144822060927,16.0,6.0,3.0,32.0,NaN,1310 Pacific Av,1310 pacific ave,2016


### Merge on address, year, using .join

In [34]:
pc = df_permits_clean.columns
ac = df_assess_clean.columns
common_cols = [item in pc for item in ac]
ac[common_cols]

Index(['Year', 'Block', 'Lot', 'Street Number', 'Street Name', 'Street Suffix',
       'Unit', 'SingleLine'],
      dtype='object')

In [33]:
#drop rows from both where either address OR year is Null

Index(['Year', 'Block', 'Lot', 'Street Number', 'Street Name', 'Street Suffix',
       'Unit', 'SingleLine'],
      dtype='object')

In [77]:
merge_cols = ['Year','SingleLine']
small_permit = df_permits_clean.iloc[:].dropna(axis=0, subset=merge_cols)
small_assess = df_assess_clean.iloc[:].dropna(subset=merge_cols)
small_merge = pd.merge(left = small_permit,right=small_assess, on=merge_cols, how='inner')

In [78]:
small_merge.columns

Index(['Permit Number', 'Permit Type', 'Permit Type Definition', 'Block_x',
       'Lot_x', 'Street Number_x', 'Street Number Suffix', 'Street Name_x',
       'Street Suffix_x', 'Unit_x', 'Unit Suffix', 'Description',
       'Current Status', 'Current Status Date', 'Filed Date', 'Issued Date',
       'Completed Date', 'First Construction Document Date',
       'Structural Notification', 'Number of Existing Stories',
       'Number of Proposed Stories', 'Voluntary Soft-Story Retrofit',
       'Fire Only Permit', 'Permit Expiration Date', 'Estimated Cost',
       'Revised Cost', 'Existing Use', 'Existing Units', 'Proposed Use',
       'Proposed Units', 'Plansets', 'TIDF Compliance',
       'Existing Construction Type', 'Existing Construction Type Description',
       'Proposed Construction Type', 'Proposed Construction Type Description',
       'Site Permit', 'Supervisor District',
       'Neighborhoods - Analysis Boundaries', 'Zipcode', 'Location',
       'Record ID', 'SF Find Neighborh

In [72]:
small_merge.set_index('Permit Number', inplace=True)

In [76]:
[print(entry) for entry in small_merge['SingleLine'].head(100)]

242 ashton ave
242 ashton ave
242 ashton ave
274 victoria st
274 victoria st
1310 pacific ave
1310 pacific ave
1751 silver ave
1751 silver ave
1762 11th ave
80 oliver st
80 oliver st
89 norton st
1741 16th ave
1741 16th ave
1741 16th ave
2558 21st ave
224 brazil ave
676 capp st
676 capp st
78 rotteck st
78 rotteck st
601 42nd ave
601 42nd ave
510 22nd ave
842 monterey bl
2238 kirkham st
1922 48th ave
476 39th ave
3817 26th st
3817 26th st
3817 26th st
3817 26th st
3125 lincoln way
3125 lincoln way
721 ashbury st
721 ashbury st
721 ashbury st
721 ashbury st
721 ashbury st
721 ashbury st
721 ashbury st
721 ashbury st
95 laidley st
1698 27th ave
1698 27th ave
194 maddux ave
194 maddux ave
194 maddux ave
366 sussex st
366 sussex st
366 sussex st
215 kenwood way
215 kenwood way
1053 tennessee st
1053 tennessee st
1053 tennessee st
1053 tennessee st
227 randolph st
227 randolph st
227 randolph st
427 joost ave
427 joost ave
427 joost ave
368 princeton st
368 princeton st
368 princeton st
96 

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

# Remove Unneeded Columns & save

In [102]:
path_pickle = './Pickled/merged_permit_assess.pkl'
# small_merge.to_pickle(path_pickle)
permit_assess = pd.read_pickle(path_pickle)

In [103]:
permit_assess.columns

Index(['Permit Number', 'Permit Type', 'Permit Type Definition', 'Block_x',
       'Lot_x', 'Street Number_x', 'Street Number Suffix', 'Street Name_x',
       'Street Suffix_x', 'Unit_x', 'Unit Suffix', 'Description',
       'Current Status', 'Current Status Date', 'Filed Date', 'Issued Date',
       'Completed Date', 'First Construction Document Date',
       'Structural Notification', 'Number of Existing Stories',
       'Number of Proposed Stories', 'Voluntary Soft-Story Retrofit',
       'Fire Only Permit', 'Permit Expiration Date', 'Estimated Cost',
       'Revised Cost', 'Existing Use', 'Existing Units', 'Proposed Use',
       'Proposed Units', 'Plansets', 'TIDF Compliance',
       'Existing Construction Type', 'Existing Construction Type Description',
       'Proposed Construction Type', 'Proposed Construction Type Description',
       'Site Permit', 'Supervisor District',
       'Neighborhoods - Analysis Boundaries', 'Zipcode', 'Location',
       'Record ID', 'SF Find Neighborh

In [104]:
drop_cols = ['Block_x',
       'Lot_x', 'Street Number_x', 'Street Number Suffix', 'Street Name_x',
       'Street Suffix_x', 'Unit_x', 'Unit Suffix', 'Property Location', 'Parcel Number', 'Block_y', 'Lot_y',
         'Street Number_y', 'Street Name_y', 'Street Suffix_y', 'Unit_y']

In [105]:
permit_assess.drop(drop_cols, axis=1, inplace=True)

In [106]:
mostly_na = permit_assess.isna().sum().to_frame()

#remove any column with more than 1/3 missing data. Make sure completed date isn't here (needed for approval time!)
na_cols = list(mostly_na[mostly_na[0]>permit_assess.shape[0]/3].index.values)

In [107]:
permit_assess.drop(na_cols, axis=1, inplace=True)

### Filter out reeeeeally slow approvals (>4years)

In [108]:
365*4

1460

In [109]:
year_filter = permit_assess['approval_in_days']<pd.to_timedelta('1461 days 00:00:00.00000')

In [110]:
permit_assess = permit_assess[year_filter]

In [111]:
permit_assess.shape

(124502, 65)

In [112]:
permit_assess.dtypes.to_frame().transpose()

,Permit Number,Permit Type,Permit Type Definition,Description,Current Status,Current Status Date,Filed Date,Issued Date,Completed Date,Number of Existing Stories,Number of Proposed Stories,Permit Expiration Date,Estimated Cost,Revised Cost,Existing Use,Existing Units,Proposed Use,Proposed Units,Plansets,Existing Construction Type,Existing Construction Type Description,Proposed Construction Type,Proposed Construction Type Description,Supervisor District,Neighborhoods - Analysis Boundaries,Zipcode,Location,Record ID,SF Find Neighborhoods,Current Police Districts,Current Supervisor Districts,Analysis Neighborhoods,approval_in_days,Address,SingleLine,Year,assess_id,Volume Number,Use Code,Use Definition,Property Class Code,Property Class Code Definition,Number of Bathrooms,Number of Bedrooms,Number of Rooms,Number of Stories,Number of Units,Zoning Code,Construction Type,Lot Depth,Lot Frontage,Property Area,Basement Area,Lot Area,Tax Rate Area Code,Percent of Ownership,Misc Exemption Value,Homeowner Exemption Value,Assessed Fixtures Value,Assessed Improvement Value,Assessed Land Value,Assessed Personal Property Value,Assessor Neighborhood District,Assessor Neighborhood Code,Assessor Neighborhood
0,object,int64,object,object,object,object,datetime64[ns],object,object,float64,float64,object,float64,float64,object,float64,object,float64,float64,float64,object,object,object,float64,object,float64,object,int64,float64,float64,float64,float64,object,object,object,int64,int64,int64,object,object,object,object,float64,int64,int64,int64,int64,object,object,float64,float64,int64,int64,float64,float64,float64,int64,int64,int64,int64,int64,int64,float64,object,object


### filter out only completed projects - not needed

In [113]:
permit_assess['Current Status'].value_counts()

complete      93006
issued        23296
expired        6679
cancelled      1010
reinstated      245
suspend         114
withdrawn        91
revoked          58
approved          2
denied            1
Name: Current Status, dtype: int64

### delta_stories and delta_units

In [114]:
permit_assess['delta_stories'] = permit_assess['Number of Proposed Stories'] - permit_assess['Number of Existing Stories']

In [115]:
permit_assess['delta_units'] = permit_assess['Proposed Units'] - permit_assess['Existing Units']

In [116]:

permit_assess['Issued Date'] = pd.to_datetime(permit_assess['Issued Date'])

In [117]:
permit_assess['approve_days'] = (permit_assess['Issued Date'] - permit_assess['Filed Date']).dt.days

In [118]:
permit_assess.dtypes.to_frame().transpose()

,Permit Number,Permit Type,Permit Type Definition,Description,Current Status,Current Status Date,Filed Date,Issued Date,Completed Date,Number of Existing Stories,Number of Proposed Stories,Permit Expiration Date,Estimated Cost,Revised Cost,Existing Use,Existing Units,Proposed Use,Proposed Units,Plansets,Existing Construction Type,Existing Construction Type Description,Proposed Construction Type,Proposed Construction Type Description,Supervisor District,Neighborhoods - Analysis Boundaries,Zipcode,Location,Record ID,SF Find Neighborhoods,Current Police Districts,Current Supervisor Districts,Analysis Neighborhoods,approval_in_days,Address,SingleLine,Year,assess_id,Volume Number,Use Code,Use Definition,Property Class Code,Property Class Code Definition,Number of Bathrooms,Number of Bedrooms,Number of Rooms,Number of Stories,Number of Units,Zoning Code,Construction Type,Lot Depth,Lot Frontage,Property Area,Basement Area,Lot Area,Tax Rate Area Code,Percent of Ownership,Misc Exemption Value,Homeowner Exemption Value,Assessed Fixtures Value,Assessed Improvement Value,Assessed Land Value,Assessed Personal Property Value,Assessor Neighborhood District,Assessor Neighborhood Code,Assessor Neighborhood,delta_stories,delta_units,approve_days
0,object,int64,object,object,object,object,datetime64[ns],datetime64[ns],object,float64,float64,object,float64,float64,object,float64,object,float64,float64,float64,object,object,object,float64,object,float64,object,int64,float64,float64,float64,float64,object,object,object,int64,int64,int64,object,object,object,object,float64,int64,int64,int64,int64,object,object,float64,float64,int64,int64,float64,float64,float64,int64,int64,int64,int64,int64,int64,float64,object,object,float64,float64,int64


In [119]:
path_pickle = './Pickled/ml_ready.pkl'
permit_assess.to_pickle(path_pickle)
ml_ready = pd.read_pickle(path_pickle)

In [120]:
ml_ready

,Permit Number,Permit Type,Permit Type Definition,Description,Current Status,Current Status Date,Filed Date,Issued Date,Completed Date,Number of Existing Stories,Number of Proposed Stories,Permit Expiration Date,Estimated Cost,Revised Cost,Existing Use,Existing Units,Proposed Use,Proposed Units,Plansets,Existing Construction Type,Existing Construction Type Description,Proposed Construction Type,Proposed Construction Type Description,Supervisor District,Neighborhoods - Analysis Boundaries,Zipcode,Location,Record ID,SF Find Neighborhoods,Current Police Districts,Current Supervisor Districts,Analysis Neighborhoods,approval_in_days,Address,SingleLine,Year,assess_id,Volume Number,Use Code,Use Definition,Property Class Code,Property Class Code Definition,Number of Bathrooms,Number of Bedrooms,Number of Rooms,Number of Stories,Number of Units,Zoning Code,Construction Type,Lot Depth,Lot Frontage,Property Area,Basement Area,Lot Area,Tax Rate Area Code,Percent of Ownership,Misc Exemption Value,Homeowner Exemption Value,Assessed Fixtures Value,Assessed Improvement Value,Assessed Land Value,Assessed Personal Property Value,Assessor Neighborhood District,Assessor Neighborhood Code,Assessor Neighborhood,delta_stories,delta_units,approve_days
3,201707242676,8,otc alterations permit,"replace (11) windows not visible from street, ...",complete,08/28/2017,2017-07-24,2017-07-24,08/28/2017,2.0,2.0,07/19/2018,12804.0,12804.0,1 family dwelling,1.0,1 family dwelling,1.0,0.0,5.0,wood frame (5),5,wood frame (5),7.0,Oceanview/Merced/Ingleside,94112.0,"(37.722633125765405, -122.46195088671291)",1471551214718,64.0,10.0,8.0,24.0,0 days 00:00:00,242 Ashton Av,242 ashton ave,2017,1803809,41,SRES,Single Family Residential,D,Dwelling,2.0,0,5,1,1,RH1,D,0.0,0.0,1250,0,2809.0,1000.0,1.0,0,7000,0,119595,60676,0,3.0,3H,Ingleside,0.0,0.0,0
4,201708084132,8,otc alterations permit,replace 6 windows visible from street; no stru...,complete,01/18/2018,2017-08-08,2017-08-08,01/18/2018,2.0,2.0,08/03/2018,8550.0,8550.0,1 family dwelling,1.0,1 family dwelling,1.0,0.0,5.0,wood frame (5),5,wood frame (5),7.0,Oceanview/Merced/Ingleside,94112.0,"(37.722633125765405, -122.46195088671291)",1473665214718,64.0,10.0,8.0,24.0,0 days 00:00:00,242 Ashton Av,242 ashton ave,2017,1803809,41,SRES,Single Family Residential,D,Dwelling,2.0,0,5,1,1,RH1,D,0.0,0.0,1250,0,2809.0,1000.0,1.0,0,7000,0,119595,60676,0,3.0,3H,Ingleside,0.0,0.0,0
6,201608296300,3,additions alterations or repairs,comply with nov 201625382 permit for furnace/w...,issued,05/30/2019,2016-08-29,2019-05-30,NaN,2.0,2.0,05/24/2020,21000.0,45000.0,1 family dwelling,1.0,2 family dwelling,2.0,2.0,5.0,wood frame (5),5,wood frame (5),11.0,Oceanview/Merced/Ingleside,94132.0,"(37.71566908950004, -122.46503924884911)",1554232514844,65.0,10.0,1.0,24.0,1004 days 00:00:00,274A Victoria St,274 victoria st,2016,2018919,42,SRES,Single Family Residential,D,Dwelling,1.0,0,5,1,1,RH1,D,0.0,0.0,1255,0,2495.0,1000.0,0.5,0,7000,0,180566,421328,0,3.0,3G,Ingleside Heights,0.0,1.0,1004
8,201609157815,8,otc alterations permit,replace (e) brick foundation in kind with conc...,complete,01/22/2018,2016-09-15,2016-10-06,01/22/2018,2.0,2.0,10/01/2017,40000.0,88400.0,1 family dwelling,1.0,1 family dwelling,1.0,2.0,5.0,wood frame (5),5,wood frame (5),3.0,Russian Hill,94109.0,"(37.795767061509025, -122.41710075059225)",143753060927,16.0,6.0,3.0,32.0,21 days 00:00:00,1310 Pacific Av,1310 pacific ave,2016,1831052,2,MRES,Multi-Family Residential,FS,Flat & Store 4 units or less,1.0,0,8,2,2,NC2,D,0.0,0.0,2332,0,1197.0,1000.0,1.0,0,0,0,127796,42512,0,8.0,8C,Nob Hill,0.0,0.0,21
10,201610210942,3,additions alterations or repairs,unit legalization ord #43-14. interior alterat...,issued,01/17/2019,2016-10-21,2019-01-17,NaN,2.0,2.0,01/01/2022,40000.0,240000.0,1 family dwelling,1.0,2 family dwelling,2.0,2.0,5.0,wood frame (5),5,wood frame (5),10.0,Bayview Hunters Point,94124.0,"(37.73391776288098, -122.4032854512375)",1538268513959,87.0,2.0,9.0,1.0,818 days 00:00:00,1751